In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from mol_td.utils import load_config
from mol_td.data_fns import load_data, prep_data, get_split, prep_data_torch
import jax
from typing import Any, Callable, Sequence, Optional
from jax import lax, random as rnd, numpy as jnp
import flax
from flax.core import freeze, unfreeze
from flax import linen as nn
import optax
import wandb
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset



In [2]:
cfg = load_config('/home/amawi/projects/mol-td/configs/default_config.yaml')
cfg

{'WANDB': {'user': 'xmax1'},
 'TRAIN': {'n_epochs': 1000, 'lr': 0.001},
 'PATHS': {'root': '/home/amawi/projects/mol-td',
  'data': './data',
  'results': './results/test',
  'default_config': './configs/default_config.yaml',
  'uracil_xyz': './data/uracil.xyz'}}

In [4]:
# load and prep the data
data, raw_data = load_data('/home/amawi/projects/mol-td/data/uracil_dft.npz')

# train_loader, val_loader, test_loader = prep_data(data, data)



(133770, 12, 3) (133770, 12, 3) (133770, 12, 1)


In [5]:
train_loader = prep_data_torch(data, data)

2022-04-03 17:37:07.785463: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/amawi/.conda/envs/td/lib/python3.10/site-packages/jax/_src/random.py:371: FutureWarning: jax.random.shuffle is deprecated and will be removed in a future release. Use jax.random.permutation with independent=True.
  warnings.warn(msg, FutureWarning)


TypeError: 'int' object is not callable

In [9]:
batch_size = 32
n_batches = len(val_idxs) // batch_size
n_remainder = len(val_idxs) - n_batches * batch_size
val_idxs = val_idxs[:-n_remainder]
val_idxs = val_idxs.reshape((n_batches, batch_size))
print(val_idxs.shape)



# train_loader = [data[train_idxs], data[train_idxs]]

(627, 32)


In [12]:
list(data.keys())

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']

In [ ]:
# initialise the model




In [ ]:
# test a forward pass with wandb


def test_train(model, params, train_loader, val_loader=None, cfg=None):

    if cfg is None: cfg = load_config()  # loads default

    run = wandb.init(project=cfg['PATHS']['results'], entity=cfg['WANDB']['user'], config=cfg['TRAIN'])

    loss_grad_fn = jax.value_and_grad(loss_fn)

    tx = optax.sgd(learning_rate=cfg['TRAIN']['lr'])
    opt_state = tx.init(params)

    for epoch in range(cfg['TRAIN']['n_epochs']):
        for batch_idx, (batch, target) in enumerate(train_loader):
            
            loss, grads = loss_grad_fn(params, batch, target)
            updates, opt_state = tx.update(grads, opt_state)
            params = optax.apply_updates(params, updates)

            wandb.log({'loss': loss})

            # indicators TODO

        if val_loader is not None:
            for batch_idx, (batch, target) in enumerate(train_loader):

                val_loss = loss_fn(batch)

    run.finish()